## 🛠️ Mod5 Data Challenge 6: Segmentation & RFM


**Why this activity?**  
You will practice building **RFM segmentation** from real e-commerce transactions to answer: *Who are our best customers, who is at risk, and who should we win back?*

**Dataset:** UCI Online Retail II (Excel, 2009–2011). 

**Goals — You will be able to:**
1) Load and minimally prepare real transaction data for RFM.  
2) Compute **Recency, Frequency, Monetary** per customer.  
3) Score and label simple **RFM segments**.  
4) Interpret segments into **business actions**.

**Interview practice:**
- **Q1:** Give two concrete actions you’d take from an RFM analysis.  
- **Q2:** How would you adapt RFM for a product with no revenue (e.g., a music app)?



### 👩‍🏫 Instructor-Led Demo (20 minutes)

### Step 1 — Load the Online Retail II Excel and prepare fields

We will:
1) Read both sheets from the Excel file and concatenate.  
2) Parse dates.  
3) Drop rows with missing **CustomerID** (needed for RFM).  
4) Create **activity_date** alias from `InvoiceDate`.  
5) Create **line_total = Quantity * UnitPrice** (we’ll use positive line totals for Monetary).

**Speaker Notes:**  
We keep setup minimal. For **Monetary**, we’ll use **positive line totals only** to avoid refunds/returns making negative spend dominate segments. In production, you’d decide whether to net refunds or keep separate KPIs.



In [1]:
# import packages per usual 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import timedelta

In [2]:
#YOU'VE SEEN THIS CODE ALREADY :) 

# Path to your downloaded Excel file
DATA_PATH = '/Users/Marcy_Student/Desktop/Marcy-Modules/marcy-git/DA2025_Lectures/Mod5/DataChallenges/data/online_retail_II.xlsx'

# 1) Read both sheets and concatenate
sheets = ["Year 2009-2010", "Year 2010-2011"]
df_list = [pd.read_excel(DATA_PATH, sheet_name=s, engine="openpyxl") for s in sheets]
df = pd.concat(df_list, ignore_index=True)

# 2) Parse datetime
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"], errors="coerce")

# 3) Drop missing CustomerID (required to build RFM)
df = df.dropna(subset=["Customer ID"]).copy()
df["Customer ID"] = df["Customer ID"].astype(int)

# 4) Activity date alias
df["activity_date"] = df["InvoiceDate"]

# 5) Line total (Monetary basis uses positive spend only)
df["line_total"] = df["Quantity"] * df["Price"]
snapshot_date = df["InvoiceDate"].max().normalize() + pd.Timedelta(days=1)

df.head()


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,activity_date,line_total
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom,2009-12-01 07:45:00,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,2009-12-01 07:45:00,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,2009-12-01 07:45:00,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom,2009-12-01 07:45:00,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom,2009-12-01 07:45:00,30.0


### Step 2 — Select a market and compute RFM
We will:  
1) Filter to `Country == "United Kingdom"`  
2) Compute:  
   - **Recency**: days since last invoice  
   - **Frequency**: # of unique invoices per customer  
   - **Monetary**: total revenue per customer (can be negative if net refunds)

**Speaker Notes:**  
Frequency as **unique invoices** avoids inflating frequent line items on the same order.


In [4]:
df_uk = df[df['Country'] == 'United Kingdom']

rfm_uk = (
    df_uk.groupby("Customer ID")
          .agg(
              Recency=("InvoiceDate", lambda s: (snapshot_date - s.max()).days),
              Frequency=("Invoice", "nunique"),
              Monetary=("Price", "sum")
          )
          .reset_index()
)

rfm_uk.head()


,Customer ID,Recency,Frequency,Monetary
0,12346,325,17,580.44
1,12608,404,1,26.04
2,12745,486,2,67.98
3,12746,527,3,73.95
4,12747,2,32,1166.00


### Step 3 — Score R, F, M and label segments
We will:  
1) Score each metric into **quartiles** (1–4)  
   - **Recency**: lower is better → labels `[4,3,2,1]`  
   - **Frequency/Monetary**: higher is better → labels `[1,2,3,4]`  
2) Combine to an **RFM_Score** (e.g., 443)  
3) Map simple labels (Champions, Loyal, At‑Risk, Lost)

**Speaker Notes:**  
Label rules are business‑defined. Keep logic simple and transparent.


In [5]:
# Avoid errors if too few unique values for qcut:
def safe_qcut(x, q, labels):
    try:
        return pd.qcut(x.rank(method="first"), q, labels=labels)
    except ValueError:
        # fallback: all middle score if not enough unique values
        return pd.Series([labels[len(labels)//2]] * len(x), index=x.index)

# Scores
r_scores = pd.qcut(rfm_uk["Recency"], 4, labels=[4,3,2,1])  # lower recency = better
f_scores = safe_qcut(rfm_uk["Frequency"], 4, labels=[1,2,3,4])
m_scores = pd.qcut(rfm_uk["Monetary"], 4, labels=[1,2,3,4])

rfm_uk["R_score"] = r_scores.astype(int)
rfm_uk["F_score"] = f_scores.astype(int)
rfm_uk["M_score"] = m_scores.astype(int)
rfm_uk["RFM_Score"] = r_scores.astype(int) + f_scores.astype(int) + m_scores.astype(int)

# Simple labeler
def label_rfm(row):
    r,f,m = row["R_score"], row["F_score"], row["M_score"]
    if r>=3 and f>=3 and m>=3:
        return "Champions"
    if r>=3 and f>=2:
        return "Loyal"
    if r<=2 and f>=3 and m>=3:
        return "At-Risk Loyalists"
    if r<=2 and f<=2 and m<=2:
        return "Lost"
    return "Others"

rfm_uk["Segment"] = rfm_uk.apply(label_rfm, axis=1)

rfm_uk.head()

,Customer ID,Recency,Frequency,Monetary,R_score,F_score,M_score,RFM_Score,Segment
0,12346,325,17,580.44,2,4,4,10,At-Risk Loyalists
1,12608,404,1,26.04,1,1,1,3,Lost
2,12745,486,2,67.98,1,1,2,4,Lost
3,12746,527,3,73.95,1,2,2,5,Lost
4,12747,2,32,1166.00,4,4,4,12,Champions


### Step 4 — Summarize segments & propose actions

We will:  
1) Count customers per segment  
2) Compute total Monetary by segment (optional)  
3) List **actions** per segment (e.g., loyalty, win‑back)

**Speaker Notes:**  
Tie segments to decisions (email, pricing, UX). Emphasize clarity over complexity.


In [6]:
seg_counts = rfm_uk["Segment"].value_counts().rename_axis("Segment").to_frame("Customers")
seg_value  = rfm_uk.groupby("Segment")["Monetary"].sum().to_frame("Total_Monetary").sort_values("Total_Monetary", ascending=False)

display(seg_counts)
display(seg_value)

# Example actions (as a dict, could be converted to a table)
segment_actions = {
    "Champions": "VIP perks, early access, referrals",
    "Loyal": "Tiered rewards, personalized recommendations",
    "At-Risk Loyalists": "Win-back offers, survey for friction",
    "Lost": "Exit survey, targeted reactivation",
    "Others": "Onboarding education, nudges"
}
segment_actions


,Customers
Segment,
Champions,1586
Lost,1554
Others,865
Loyal,815
At-Risk Loyalists,590


,Total_Monetary
Segment,
Champions,1676485.518
At-Risk Loyalists,457710.034
Others,220286.352
Loyal,115017.002
Lost,94904.971


{'Champions': 'VIP perks, early access, referrals',
 'Loyal': 'Tiered rewards, personalized recommendations',
 'At-Risk Loyalists': 'Win-back offers, survey for friction',
 'Lost': 'Exit survey, targeted reactivation',
 'Others': 'Onboarding education, nudges'}

### 👩‍🏫 Student Led (30 minutes) -- ANSWER KEY

### Step 1 — Use the original dataframe (`df`) above and Filter to Germany and compute RFM
 
1) Filter `Country == "Germany"`  
2) Compute Recency (days since last invoice), Frequency (# unique invoices), Monetary (sum of amount)

In [ ]:
# 1) Filter to Germany
df_de = None  

# 2) Compute RFM
rfm_de = None  


rfm_de.head()


### Step 2 — Score R, F, M and label segments

1) Create R, F, M scores (quartiles)  
2) Combine into `RFM_Score`  
3) Label segments with simple, explainable rules


In [ ]:
# Helper for qcut with few uniques
def safe_qcut(x, q, labels):
    try:
        return pd.qcut(x.rank(method="first"), q, labels=labels)
    except ValueError:
        return pd.Series([labels[len(labels)//2]] * len(x), index=x.index)

# Scores (fill in)
r_scores = None  
f_scores = None 
m_scores = None  

rfm_de["R_score"] = None  
rfm_de["F_score"] = None  
rfm_de["M_score"] = None  
rfm_de["RFM_Score"] = None  

 
 #Feel free to change the labels if you like :)
 def label_rfm(row):
   r,f,m = row["R_score"], row["F_score"], row["M_score"]
    if r>=3 and f>=3 and m>=3:
        return "Champions"
    if r>=3 and f>=2:
        return "Loyal"
    if r<=2 and f>=3 and m>=3:
        return "At-Risk Loyalists"
    if r<=2 and f<=2 and m<=2:
        return "Lost"
    return "Others"

rfm_de["Segment"] = None  
rfm_de.head(10)


### Step 3 — Summarize segments and propose actions

1) Count customers per segment  
2) Compute total Monetary per segment (optional)  
3) Suggest 1 concrete action per major segment for **Germany**


In [ ]:
seg_counts_de = None
seg_value_de  = None

display(seg_counts_de)
display(seg_value_de)

## Wrap‑Up (10 mins): Turning RFM into Action

Students:  Be prepared to compare UKs vs. Germany's RFM/insights :)

- **Explainability:** RFM is transparent—every stakeholder can understand Recency, Frequency, Monetary.
- **Actionability:** Map segments to clear plays:
  - **Champions:** VIP perks, early access, referrals
  - **Loyal:** Tiered rewards, personalized bundles
  - **At‑Risk Loyalists:** Win‑back offers, friction survey
  - **Lost:** Lightweight reactivation or exit survey
- **Caveats:** Monetary can be negative if refunds dominate—decide whether to clamp at 0 for your use case.
- **Next steps:**  
  - Compare countries or channels;  
  - Try deciles (1–10) for finer splits;  
  - Track segment movement month‑over‑month.
